In [ ]:
import numpy as np 
import pandas as pd 
import datetime

In [ ]:
bpl_df = pd.read_csv("soccer-match-event-dataset/events_England.csv")
bpl_df = bpl_df[bpl_df["eventName"] == "Pass"]
game_id = pd.read_csv("soccer-match-event-dataset/games.csv")#importing and merging
game_data_added = (bpl_df.merge(game_id, left_on='matchId', right_on='game_id'))


In [ ]:
teams = pd.read_csv('soccer-match-event-dataset/teams.csv')
teams = teams[["name","wyId"]]
player_team_added = (game_data_added.merge(teams, left_on='player_team_id', right_on='wyId'))#getting teams
player_team_added = player_team_added.rename(columns={'name':"player_team_name"})

In [ ]:
player_team_added["opposing_team_id"] = np.where(player_team_added['player_team_id'] == player_team_added['home_team_id'], player_team_added['away_team_id'], player_team_added['home_team_id'])
player_team_added = (player_team_added.merge(teams, left_on='opposing_team_id', right_on='wyId'))
player_team_added = player_team_added.rename(columns={'name':"opposing_team_name"})#cleaning
player_team_added = player_team_added.drop(columns=['wyId_y', 'wyId_x'])

In [ ]:
player_team_added = (player_team_added.merge(teams, left_on='home_team_id', right_on='wyId'))
player_team_added = player_team_added.rename(columns={"name":"home_team_name"})
fielddata = pd.read_csv('field-data/fielddata.csv')#getting field sizes
player_team_added = (player_team_added.merge(fielddata, left_on='home_team_name', right_on='Name'))
player_team_added

In [ ]:
player_team_added["orig_distogoal_y"] = player_team_added["y_field_length_meters"] * (player_team_added["pos_orig_y"]/100)
player_team_added["orig_distogoal_x"] = player_team_added["x_field_length_meters"] * (player_team_added["pos_orig_x"]/100)
player_team_added["dest_distogoal_y"] = player_team_added["y_field_length_meters"] * (player_team_added["pos_dest_y"]/100)#cleaning
player_team_added["dest_distogoal_x"] = player_team_added["x_field_length_meters"] * (player_team_added["pos_dest_x"]/100)
player_team_added["distance_travelled"] = np.sqrt((player_team_added["orig_distogoal_y"] - player_team_added["dest_distogoal_y"])**2 + (player_team_added["orig_distogoal_x"] - player_team_added["dest_distogoal_x"])**2)

In [ ]:
managers = pd.read_csv('/managers-new-2/managers.csv')#getting new managers
managers["Date Before"] = pd.to_datetime(managers["Date Before"])
managers["Date Until"] = pd.to_datetime(managers["Date Until"])
managers.dtypes
player_team_added["game_date"] = pd.to_datetime(player_team_added["game_date"])
player_team_added.shape
managers_added = player_team_added.merge(managers,
                       left_on = "player_team_name", right_on = "Team")
managers_added#cleaning

In [ ]:
conditions = [
    (managers_added['Date Before'] < managers_added["game_date"]) & (managers_added['Date Until'] >= managers_added["game_date"]),
    (managers_added['Date_Before_2'] < managers_added["game_date"]) & (managers_added['Date_Until_2'] >= managers_added["game_date"])
]
choices = [managers_added["First Manager"], managers_added["Secondary Manager"]]
managers_added['Manager'] = np.select(conditions, choices, default=managers_added["Third Manager"])
managers_added[["Manager","Date Before","Date Until","game_date","Team"]]#cleaning

In [ ]:
managers_added.columns#cleaning
managers_added = managers_added.drop(columns=['First Manager', 'Date Before',
       'Date Until', 'Secondary Manager', 'Date_Before_2', 'Date_Until_2',
       'Third Manager'])


In [ ]:
managers_added.columns
new_important = managers_added[['matchId','game_id','distance_travelled', 'Team', 'Manager','home_team_name','Stadium']]
fielddata = pd.read_csv('field-data/fielddata.csv')#more field data
more_new_important = new_important.merge(fielddata, left_on = "Stadium", right_on = "Stadium")
more_new_important

In [ ]:
by_game = pd.DataFrame(columns=["game_id","distance_travelled","Team","Manager","home_team_name","Stadium","y_field_length_meters","x_field_length_meters"])
arsenetest = more_new_important[more_new_important["Manager"] == "Arsene Wenger"]
np.mean(arsenetest["distance_travelled"])

In [ ]:
game_dict = {}
for i in ["game_id","distance_travelled","Team","Manager","home_team_name","Stadium","y_field_length_meters","x_field_length_meters"]:
    game_dict[i] = []
game_dict
for game in more_new_important["game_id"].unique():
    #getting distance travelled for each manager
    dfgame = more_new_important.loc[more_new_important["game_id"] == game].reset_index()
    teams = dfgame["Team"].unique()
    team_1 = teams[0]
    team_2 = teams[1]
    dfteam_1 = dfgame.loc[dfgame["Team"] == team_1].reset_index()
    dfteam_2 = dfgame.loc[dfgame["Team"] == team_2].reset_index()
    for j in ["game_id","home_team_name","Stadium","y_field_length_meters","x_field_length_meters"]:
        game_dict[j].append(dfgame[j][0])
        game_dict[j].append(dfgame[j][0])
    game_dict["Team"].append(team_1)
    game_dict["Manager"].append(dfteam_1["Manager"][0])
    game_dict["distance_travelled"].append(np.mean(dfteam_1["distance_travelled"]))
    print(dfteam_1["Manager"][0])
    print(np.mean(dfteam_1["distance_travelled"]))
    game_dict["Team"].append(team_2)
    game_dict["Manager"].append(dfteam_2["Manager"][0])
    game_dict["distance_travelled"].append(np.mean(dfteam_2["distance_travelled"]))
    #print(game)
print(game_dict)
    
    

In [ ]:
per_game = pd.DataFrame.from_dict(game_dict)
per_game.to_csv("per_game.csv")#per game stats

In [ ]:
per_manager_csv = per_game[per_game["Team"] == per_game["home_team_name"]]
per_manager_csv = per_manager_csv[["Manager","distance_travelled"]]
per_manager = per_manager_csv.groupby(by = ["Manager"]).mean()
per_manager.to_csv("per_manager.csv")#per manager stats

In [ ]:
per_game_diff = per_game.merge(fielddata, left_on = "Team",right_on = "Name")
per_game_diff = per_game_diff.rename(columns = {"y_field_length_meters_y":"y_field_length_team",
                                               "x_field_length_meters_y":"x_field_length_team"    
                                              })
per_game_diff_away = per_game_diff.drop(columns = ["Name","Stadium_y"])
per_game_diff_away#checking away differences

In [ ]:
per_game_diff = per_game_diff.rename(columns = {"y_field_length_meters_x":"y_field_length_arena",
                                               "x_field_length_meters_x":"x_field_length_arena"    
                                              })
per_game_diff = per_game_diff[per_game_diff["Team"] != per_game_diff["home_team_name"]]
per_game_diff["Change In Length"] = per_game_diff["y_field_length_arena"]  - per_game_diff["y_field_length_team"]
per_game_diff["Change In Width"] = per_game_diff["x_field_length_arena"]  - per_game_diff["x_field_length_team"]
per_game_diff

In [ ]:
per_game_diff_analysis = per_game_diff[["Manager","distance_travelled","Change In Length","Change In Width"]]
longer_field = per_game_diff_analysis[per_game_diff_analysis["Change In Length"] > 0]
shorter_field = per_game_diff_analysis[per_game_diff_analysis["Change In Length"] < 0]
wider_field = per_game_diff_analysis[per_game_diff_analysis["Change In Width"] > 0]
narrower_field = per_game_diff_analysis[per_game_diff_analysis["Change In Width"] < 0]#getting changes
per_manager_longer = longer_field.groupby(by = ["Manager"]).mean()
per_manager_shorter = shorter_field.groupby(by = ["Manager"]).mean()
per_manager_wider = wider_field.groupby(by = ["Manager"]).mean()
per_manager_narrower = narrower_field.groupby(by = ["Manager"]).mean()
per_manager_shorter

In [ ]:
per_manager["distance_field_longer"] = per_manager_longer[["distance_travelled"]]
per_manager["distance_field_shorter"] = per_manager_shorter[["distance_travelled"]]
per_manager["distance_field_wider"] = per_manager_wider[["distance_travelled"]]
per_manager["distance_field_narrower"] = per_manager_narrower[["distance_travelled"]]
per_manager = per_manager.rename(columns={"distance_travelled":"distance_travelled_home"})#getting changes in distance
per_manager.head(10)

In [ ]:
per_manager.to_csv("per_manager_complete.csv")#final df